In [1]:
import datetime as dt
import pandas as pd

In [2]:
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas105semanas.parquet")

In [3]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)


df["client_name"] = df["client_name_inv"].str.split("] ", expand=True)[1].str.strip()



#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])

df = df.drop(columns=["origin"])

df["product_ref"] = df["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df["product_nom"] = df["product_name"].str.extract(r"\]\s*([A-Z]\w+)")


df["date_invoice"] = pd.to_datetime(df["date_invoice"])
df["mes"] = df["date_invoice"].dt.month
df["año"] = df["date_invoice"].dt.year
df["dia"] = df["date_invoice"].dt.day

df = df[~(df["product_ref"].isna())]

pd.set_option('display.max_columns', None)

df['marca'] = df['product_ref'].apply(
    lambda x: 'BALDWIN' if str(x).endswith('125') else
              'DONSSON' if str(x).endswith('025') else
              'AUT*PARTS' if str(x).endswith('189') else
              'RACOR BRASIL' if str(x).endswith('137') else
              'RACOR USA' if str(x).endswith('138') else
              'OTRA'
)

In [24]:
df.sample(10)

,product_id,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv,Sucursal_tipo,client_name,origen,Bodega,product_ref,product_nom,mes,año,dia,marca
169342,"[17680, '[DAB04772025] DA4772 FILTRO AIRE INTE...",42200.0,1430430,2.0,[DAB04772025] DA4772 FILTRO AIRE INTERNO BOBCA...,FCOT22081,2024-12-10 12:29:15,FCOT22081,389658,PRINCIPAL COTA,WILMER GIL,EXTERNOS,27071,[CC46450716] RUTH ADRIANA LINARES CABRA,VENDEDOR EXTERNO,RUTH ADRIANA LINARES CABRA,BD1,PRINCIPAL COTA,DAB04772025,DA4772,12,2024,10,DONSSON
136952,"[19225, '[BCS00263125] GS263 FILTROCOMB. ISUZU...",120400.0,1469497,4.0,"[BCS00263125] GS263 FILTROCOMB. ISUZU, CHEV. N...",PV2E37285 Mostrador Calle 6 T2/19558,2025-02-11 21:33:29,PV2E37285,399634,SUCURSAL CALLE 6,JAIME ANDRES CABALLERO CABALLERO,SUCURSALES / Ventas Mostrador Cll6,42293,[CO9016333095] SURTILLANOS AGRÍCOLA S.A.S.,MOSTRADOR,SURTILLANOS AGRÍCOLA S.A.S.,<NA>,SUCURSAL CALLE 6,BCS00263125,GS263,2,2025,11,BALDWIN
328198,"[17452, '[DAE04066025] DA4066 FILTRO AIRE SEGU...",56400.0,1239660,2.0,[DAE04066025] DA4066 FILTRO AIRE SEGURIDAD - B...,FBAQ9574,2024-01-25 12:56:11,FBAQ9574,337770,SUCURSAL BARRANQUILLA,MIGUEL ANGEL NIEBLES CERA,SUCURSALES / BARRANQUILLA / EXTERNOS,26719,[CO9008246025] TODO TRAILER DEL NORTE SAS,VENDEDOR EXTERNO,TODO TRAILER DEL NORTE SAS,BD6,SUCURSAL BARRANQUILLA,DAE04066025,DA4066,1,2024,25,DONSSON
127824,"[17066, '[DAR02594025] DA2594 FILTRO AIRE CAMI...",41100.0,1480125,1.0,[DAR02594025] DA2594 FILTRO AIRE CAMION KIA 35...,PV3E17215 Mostrador Valladolid T2/13673,2025-02-26 18:15:29,PV3E17215,402525,SUCURSAL VALLADOLID,SEBAS RODRIGUEZ,SUCURSALES / Ventas Mostrador Valladolid,17415,[CO9011799904] TODOFILTROS DEL OCCIDENTE S.A.S.,MOSTRADOR,TODOFILTROS DEL OCCIDENTE S.A.S.,<NA>,SUCURSAL VALLADOLID,DAR02594025,DA2594,2,2025,26,DONSSON
66487,"[17038, '[DAB02547025] DA2547 FILTROAIRE 1_ D...",60200.0,1553089,1.0,[DAB02547025] DA2547 FILTROAIRE 1_ DONSSON -K...,PV4E15288 Mostrador Norte T1/17394,2025-06-17 13:21:59,PV4E15288,421756,SUCURSAL NORTE,WILMER MULATO,SUCURSALES / Ventas Mostrador Norte,14685,[CO9006740741] INGENIERIA Y TRANSPORTES LAMD S...,MOSTRADOR,INGENIERIA Y TRANSPORTES LAMD S A S,<NA>,SUCURSAL NORTE,DAB02547025,DA2547,6,2025,17,DONSSON
281018,"[17058, '[DAB02570025] DA2570 FILTRO AIRE DONS...",39000.0,1296659,1.0,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,FMED10657,2024-04-29 13:58:00,FMED10657,353366,SUCURSAL MEDELLIN,WILSON PELAEZ,SUCURSALES / MEDELLÍN / EXTERNOS MED,37612,[CO9008443832] SERVICIOS Y LOGISTICA HA S.A.S.,VENDEDOR EXTERNO,SERVICIOS Y LOGISTICA HA S.A.S.,BD5,SUCURSAL MEDELLIN,DAB02570025,DA2570,4,2024,29,DONSSON
85525,"[19374, '[BCS00484125] GS484 FILTRO COMBUST. S...",111400.0,1531295,2.0,[BCS00484125] GS484 FILTRO COMBUST. SEPARADOR ...,FCOT24444,2025-05-14 13:13:58,FCOT24444,415770,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,3020,[CO8305152261] VISUM LTDA,VENDEDOR EXTERNO,VISUM LTDA,BD1,PRINCIPAL COTA,BCS00484125,GS484,5,2025,14,BALDWIN
367539,"[16932, '[DAR02317025] DA2317 FILTRO AIRE 1_ C...",59500.0,1192674,1.0,[DAR02317025] DA2317 FILTRO AIRE 1_ CATERPILLA...,PV7E3574 Mostrador Bucaramanga T1/4985,2023-10-23 20:54:58,PV7E3574,324962,SUCURSAL BUCARAMANGA,EDISSON MIGUEL CIFUENTES OSORIO,SUCURSALES / Sucursal Bucaramanga / MOSTRADOR ...,33076,[CC18913961] JULIO CESAR SIERRA PEÑALOZA,MOSTRADOR,JULIO CESAR SIERRA PEÑALOZA,<NA>,SUCURSAL BUCARAMANGA,DAR02317025,DA2317,10,2023,23,DONSSON
188518,"[47442, '[DCS00035189] GS035 FILTRO COMBUSTIBL...",81000.0,1407413,6.0,[DCS00035189] GS035 FILTRO COMBUSTIBLE - CUMMI...,FBUC5283,2024-11-02 15:44:04,FBUC5283,383558,SUCURSAL BUCARAMANGA,FABIAN BLANCO,SUCURSALES / Sucursal Bucaramanga / EXTERNOS B...,34288,[CC1091666583] SERGIO ALFONSO TORRADO PELAEZ,VENDEDOR EXTERNO,SERGIO ALFONSO TORRADO PELAEZ,BD7,SUCURSAL BUCARAGAMNGA,DCS00035189,GS035,11,2024,2,AUT*PARTS
154880,"[49563, '[DAB08220025

In [5]:
from datetime import datetime

# Fecha y hora actual en formato YYYYMMDD_HHMMSS
timestamp = datetime.now().strftime("%Y%m%d")

# Nombre dinámico
#filename = f"/home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_{timestamp}.xlsx"

# Guardar el Excel
#df.to_excel(filename, index=False)

#print(f"Archivo guardado: {filename}")

#Tiempo promedio 3 minutos

## VISUM

In [28]:
clientes = {"VISUM LTDA",
"VILMAR LUBRICANTES SAS",
"ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.", 
"SODATRANS LTDA",
"ESTACION Y LUBRICANTES CASABLANCA S.A.S"}

fecha_inicial = "09/1/2024"
fecha_final = "09/30/2025"

df_h = df[(df["client_name"].isin(clientes)) & (df["date_invoice"] >= fecha_inicial) & (df["date_invoice"] <= fecha_final) & (df["salesperson_name"] == "YEIS OSWALDO MUÑOZ GUERRERO")  ]

df_h = df_h.drop(columns=["product_id","id_x","id_y","partner_id_num","client_name_inv","invoice_name","origen","number"])

In [29]:
df_h["date_invoice"].max()

Timestamp('2025-09-19 14:59:01')

In [30]:
df_h.sample(10)

,price_subtotal,quantity,product_name,date_invoice,store_name,salesperson_name,sales_team_name,Sucursal_tipo,client_name,Bodega,product_ref,product_nom,mes,año,dia,marca
187484,192400.0,2.0,[BLE00709125] G709 FILTRO ACEITE PERKINS SERIE...,2024-11-06 14:12:53,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BLE00709125,G709,11,2024,6,BALDWIN
189768,151800.0,2.0,[BCS00216125] GS216 FILTRO COMBUSTIBLE JOHN DE...,2024-10-31 16:08:44,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BCS00216125,GS216,10,2024,31,BALDWIN
51711,59200.0,1.0,[BLS00459125] GS459 FILTRO ACEITE JCB 320/0413...,2025-07-15 13:59:27,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.,PRINCIPAL COTA,BLS00459125,GS459,7,2025,15,BALDWIN
89213,99800.0,2.0,[DAR08048025] DA8048 FILTRO AIRE JAC 1035 (025...,2025-05-07 20:28:37,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,DAR08048025,DA8048,5,2025,7,DONSSON
116236,492000.0,6.0,[DAB08185025] DA8185 FILTRO AIRE CARGADOR SDLG...,2025-03-19 13:39:25,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,DAB08185025,DA8185,3,2025,19,DONSSON
133141,396000.0,4.0,[BLS00202125] GS202 FILTRO ACEITE FILTROFILTRO...,2025-02-18 13:23:28,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,ESTACION Y LUBRICANTES CASABLANCA S.A.S,PRINCIPAL COTA,BLS00202125,GS202,2,2025,18,BALDWIN
63448,187200.0,1.0,[BAB04926125] DA4926 FILTRO AIRE INTERNO IHC 4...,2025-06-24 13:11:09,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BAB04926125,DA4926,6,2025,24,BALDWIN
187473,86200.0,2.0,[BCS00163125] GS163 FILTRO COMBUSTIBLE CUMMINS...,2024-11-06 14:12:53,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,VISUM LTDA,PRINCIPAL COTA,BCS00163125,GS163,11,2024,6,BALDWIN
133187,61400.0,2.0,[DLS00136189] GS136 FILTRO ACEITE - ISUZU - CA...,2025-02-18 13:20:36,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.,PRINCIPAL COTA,DLS00136189,GS136,2,2025,18,AUT*PARTS
30930,405600.0,2.0,"[BAE08142125] DA8142 FILTRO AIRE CATERPILLAR, ...",2025-08-21 13:56:51,PRINCIPAL COTA,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Credito Norte,VENDEDOR EXTERNO,ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.,PRINCIPAL COTA,BAE08142125,DA8142,8,2025,21,BALDWIN


In [31]:
df_h.to_excel("harold_vendedor.xlsx")